# HireArt Data Analysis Exercise (Simple Spreadsheet Analysis)
The marketing ops team sent over this spreadsheet containing four years of data from a CRM system.
Account managers want to find the month they're likely to contact the most clients, so they can schedule a product upgrade announcement. 

In [1]:
import numpy as np
import pandas as pd

column_names = {'Account manager': 'manager', 'Client Name': 'client',
                'Date of Contact': 'date'}

accounts = (
    pd.read_csv("via_exercise.csv", parse_dates=['Date of Contact'])
      .rename(columns=column_names)
      .dropna(thresh=3)
      .sort_index()
)

accounts.head()

,manager,client,date
0,Aidan Pouros,"Veum, McClure and Schuster",2013-12-07
1,Aidan Pouros,"Lueilwitz, Moore and Hahn",2013-12-25
2,Aidan Pouros,Cormier LLC,2013-10-22
3,Aidan Pouros,Johnston-Schaden,2013-11-20
4,Aidan Pouros,"Kris, Shanahan and Quigley",2014-07-04


In [2]:
# Extract and count contacts by month
accounts['month'] = accounts['date'].map(lambda x: x.month)
accounts_by_m = accounts.set_index(['manager','month']).sort_index().copy()
contacts_by_m = accounts_by_m['client'].groupby(['manager', 'month']).agg(['count']).unstack(level=0)

# Get total contact numbers by month and manager
contacts_by_m['total'] = contacts_by_m['count'].sum(axis=1)
contacts_by_m

count                                           total
manager Aidan Pouros Chauncey Dach Milan Crona Rigoberto White      
month                                                               
1                  8            14          18              20    60
2                 18            13          20              13    64
3                 10            18          20              10    58
4                 10            16          14              12    52
5                 16            18          18              17    69
6                 21            17          24              15    77
7                 13            16          24              13    66
8                 15            21          25              18    79
9                 21            30          36              34   121
10                62            61          52              38   213
11                22            16          15              12    65
12                23            13          20              20    76

October has highest total contact counts throughout all manager.

---
## Which month does the team contact the greatest percentage of its clients?

In [3]:
# count total clients each manager have
contact_by_client = accounts.groupby(['manager','client'], as_index=False).size()
total_clients = contact_by_client.groupby(['manager']).count().reset_index(name = 'total_clients')
total_clients

,manager,total_clients
0,Aidan Pouros,35
1,Chauncey Dach,35
2,Milan Crona,35
3,Rigoberto White,35


In [4]:
# remove double counted contacts
accounts['yr_m'] = accounts['date'].map(lambda x: str(x.year) + '-' +  str(x.month))
accounts_by_yr_m = accounts.groupby(['manager','yr_m','client']).size().reset_index(name='contact_time')
accounts_by_yr_m.head()

,manager,yr_m,client,contact_time
0,Aidan Pouros,2013-10,Abbott Group,2
1,Aidan Pouros,2013-10,"Brown, Wyman and Grimes",2
2,Aidan Pouros,2013-10,Cormier LLC,1
3,Aidan Pouros,2013-10,Donnelly-Champlin,1
4,Aidan Pouros,2013-10,Eichmann and Sons,1


In [5]:
total_contacts = accounts_by_yr_m.groupby(['manager','yr_m']).count()['client'].reset_index(name = 'contacted_clients')
contact_counts = pd.merge(total_contacts, total_clients,on ='manager')
contact_counts['percent'] = contact_counts['contacted_clients'] / contact_counts['total_clients']
contact_counts.head()

,manager,yr_m,contacted_clients,total_clients,percent
0,Aidan Pouros,2013-10,13,35,0.371429
1,Aidan Pouros,2013-11,7,35,0.200000
2,Aidan Pouros,2013-12,6,35,0.171429
3,Aidan Pouros,2014-1,1,35,0.028571
4,Aidan Pouros,2014-10,14,35,0.400000


In [6]:
# Get the average percentage by month
contact_counts['month'] = contact_counts['yr_m'].map(lambda x: int(x.split('-')[1]))
contact_counts_by_m = contact_counts.set_index(['manager','month']).sort_index().copy()
contact_counts_by_m = contact_counts_by_m['percent'].groupby(['manager', 'month']).agg(['mean']).unstack(level=0)
contact_counts_by_m['Ave_precent'] = contact_counts_by_m['mean'].sum(axis=1)/4
contact_counts_by_m

mean                                           Ave_precent
manager Aidan Pouros Chauncey Dach Milan Crona Rigoberto White            
month                                                                     
1           0.057143      0.092857    0.128571        0.142857    0.105357
2           0.128571      0.092857    0.135714        0.092857    0.112500
3           0.071429      0.121429    0.128571        0.095238    0.104167
4           0.071429      0.100000    0.092857        0.085714    0.087500
5           0.100000      0.114286    0.114286        0.121429    0.112500
6           0.142857      0.114286    0.150000        0.107143    0.128571
7           0.085714      0.114286    0.157143        0.114286    0.117857
8           0.107143      0.142857    0.171429        0.128571    0.137500
9           0.142857      0.171429    0.207143        0.228571    0.187500
10          0.371429      0.357143    0.307143        0.214286    0.312500
11          0.150000      0.100000    0.107143        0.064286    0.105357
12          0.142857      0.092857    0.121429        0.142857    0.125000

In October, account managers are likely to contact the most clients. In average, the team contact about 31% of it's clients.